In [1]:
#%load_ext watermark
%load_ext autoreload
%autoreload 2

In [2]:
import hero
from dotenv import load_dotenv
load_dotenv()
seed = 86

/Users/rsandhu/Projects/rdi_hero/pybpr/.venv/lib/python3.14/site-packages/hero/services/data_repo.py:1811: SyntaxWarning: 'return' in a 'finally' block
  return file_resource


In [3]:
# Initialize Hero client and get patched MLflow
print("Initializing Hero ML Model Registry...")
hero_client = hero.HeroClient()
model_registry = hero_client.MLModelRegistry()
mlflow = model_registry.get_patched_mlflow()
mlflow.set_tracking_uri(model_registry.get_tracking_uri())
tracking_uri = model_registry.get_tracking_uri()
print(f"MLflow Tracking URI: {tracking_uri}\n")

Initializing Hero ML Model Registry...


2026/01/30 09:53:53 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/01/30 09:53:53 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/01/30 09:53:53 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/01/30 09:53:53 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/01/30 09:53:53 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/01/30 09:53:53 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/01/30 09:53:53 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2026/01/30 09:53:53 INFO mlflow.store.db.utils: Updating database tables
2026/01/30 09:53:53 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/01/30 09:53:53 INFO alembic.runtime.migration: Will assume non-transactional DDL.
2026/01/30 09:53:53 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/01/30 09:53:53 INFO alembic.runtime

MLflow Tracking URI: https://dev-hero.nrel.gov/ml-model-registry/api/v1/proxy/dev-recommender-system-app



In [4]:
# Set experiment

# try:
#     experiment = mlflow.set_experiment(experiment_name=experiment_name)
# except:
#     experiment_id = mlflow.create_experiment(experiment_name)
experiment_name = "[Demo] pybpr"
experiment = model_registry.read_or_create_experiment(experiment_name)
print(f"Experiment: {experiment.name}")
print(f"Experiment ID: {experiment.experiment_id}\n")

Experiment: [Demo] pybpr
Experiment ID: 132



In [ ]:
from mlflow.pyfunc import PythonModel
from preprocess import preprocess


class ModelWrapper(PythonModel):
    def __init__(self, model):
        self.model = model

    def predict(self, context, model_input, params=None):
        processed_input = preprocess(model_input)
        print(f"Processed input: {processed_input}")
        print(f"Model input: {model_input}")
        return self.model.predict(processed_input)

In [ ]:
# Define tags and parameters for the experiment, these will be logged with the run
# and can be used to filter and search runs.
tags = {
    "Project": "MLOps platform tests",
    "autologging": "false"
}
params = {
    "random_state": seed,
    "n_estimators": 100,
    "max_depth": 6,
    "max_features": 3
}

# Load the diabetes dataset and split it into training and test sets
db = load_diabetes()
X_train, X_test, y_train, y_test = train_test_split(
    db.data, db.target, random_state=seed)

# Creates a hello world data file, this is just to show how to load/save data files
os.makedirs("data", exist_ok=True)
with open("data/data.json", "w", encoding="utf-8") as f:
    json.dump({"hello": "world"}, f, indent=2)

print('ready to define eval and train')

# Evaluates the model's predictions against the true values


def evaluate(y, pred):
    rmse = np.sqrt(mean_squared_error(y, pred))
    mae = mean_absolute_error(y, pred)
    r2 = r2_score(y, pred)
    return {
        "rmse": rmse,
        "mae": mae,
        "r2": r2
    }

# Trains a model and logs it to MLflow


def train_model():
    mlflow.start_run(experiment_id=experiment.experiment_id)
    mlflow.log_artifacts("data", artifact_path="states")

    run = mlflow.active_run()
    mlflow.set_tags(tags)
    mlflow.log_params(params)
    mlflow.log_input(
        mlflow.data.from_numpy(X_train),
        context='Train')
    mlflow.log_input(
        mlflow.data.from_numpy(X_test),
        context='Eval')

    # Create and train models.
    rf = RandomForestRegressor(**params)
    rf.fit(X_train, y_train)

    # Use the model to make predictions on the test dataset.
    predictions = rf.predict(X_test)
    print(predictions)

    metrics = evaluate(y_test, predictions)
    mlflow.log_metrics(metrics)
    print(metrics)

    signature = mlflow.models.infer_signature(X_test, predictions)

    # Log the model with a wrapper
    mlflow.pyfunc.log_model(
        name=f"hero-example-{int(time.time())}",
        python_model=ModelWrapper(rf),
        code_paths=["./preprocess.py"],
        signature=signature,
        # requirements="requirements.txt", # or specify custom requirements here
    )

    # Note: The standard sklearn model logging is commented out since we're using a wrapped model above (see mlflow.pyfunc.log_model).
    # mlflow.sklearn.log_model(
    #     rf,
    #     "model",
    #     signature=signature,
    #     # requirements="requirements.txt",
    # )

    mlflow.end_run()

    # Note: not all deps captured in the requirements.txt are automatically captured by mlflow.
    # You may need to manually add deps from git or private registries using a command like this:
    # mlflow.models.update_model_requirements(
    #     model_uri=f"runs:/{run.info.run_id}/model",
    #     operation="add",
    #     requirement_list=["hero@ git+https://github.nrel.gov/Hero/hero@v0.8.0"],
    # )
    print(experiment)
    # print experiment id
    print(f"Experiment_id: {experiment.experiment_id}")
    print(f"Run ID: {run.info.run_id}")

    return run


# Do the thing and get the resulting run
run = train_model()

In [ ]:
models = model_registry.list_registered_models()

In [ ]:
uri = model_registry.get_tracking_uri()
print(f"Tracking URI: {uri}")

In [ ]:
experiments = model_registry.list_experiments()
print(f"Experiments: {experiments}")

In [ ]:
for k, v in experiments[0].items():
    print(f"{k}: {v}")

In [ ]:
created = model_registry.read_experiment(id=experiments[0]["experiment_id"])

In [ ]:
model_registry.create_experiment(name="temp_experiment_12345")

In [ ]:
model_registry.list_registered_models()